# Support Vector Machines


In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from helpers import get_scores
from sklearn.grid_search import RandomizedSearchCV
from transformers import transformer
from data_builder import load_test_data, load_dev_data, load_small_dev_data

df, target = load_dev_data()
X = transformer.fit_transform(df)



Conjunto a probar

1)C in range [0:20] Dice el manual:Por default es uno. If you have a lot of noisy observations you should decrease it. It corresponds to regularize more the estimation.

2)Kernels {'rbf ' ,' linear' ,' poly' ,'sigmoid'}  

3) para poly podemos hacer un range de degrees, esa opcion va a ser ignorada por los otros kernels asi que podemos hacer degree in range[2:5]. Podemos elegis=r ademas si agregamos un coef0, o sea el termino independiente, no veo por que seria importnte por ahora eso.

4)(REVISAR)Por default hay un parametro epsilon=0.1. Epsilon indica un rango de distancia que no sera considerado como penalidad, si el vector esta a espilon--ver

decision_function(X)[source]
Distance of the samples X to the separating hyperplane.
Parameters:	
X : array-like, shape (n_samples, n_features)
returns:	
X : array-like, shape (n_samples, n_classes * (n_classes-1) / 2)
Returns the decision function of the sample for each class in the model. If decision_function_shape=’ovr’, the shape is (n_samples, n_classes)

-Cuando fiteamos este modelo podemos elegir un sample weight, pero no creo que lo necesitemos ya que estan balanceados.



In [2]:
from sklearn.svm import SVC, LinearSVC

help(LinearSVC)

Help on class LinearSVC in module sklearn.svm.classes:

class LinearSVC(sklearn.base.BaseEstimator, sklearn.linear_model.base.LinearClassifierMixin, sklearn.feature_selection.from_model._LearntSelectorMixin, sklearn.linear_model.base.SparseCoefMixin)
 |  Linear Support Vector Classification.
 |  
 |  Similar to SVC with parameter kernel='linear', but implemented in terms of
 |  liblinear rather than libsvm, so it has more flexibility in the choice of
 |  penalties and loss functions and should scale better to large numbers of
 |  samples.
 |  
 |  This class supports both dense and sparse input and the multiclass support
 |  is handled according to a one-vs-the-rest scheme.
 |  
 |  Read more in the :ref:`User Guide <svm_classification>`.
 |  
 |  Parameters
 |  ----------
 |  C : float, optional (default=1.0)
 |      Penalty parameter C of the error term.
 |  
 |  loss : string, 'hinge' or 'squared_hinge' (default='squared_hinge')
 |      Specifies the loss function. 'hinge' is the st

In [5]:
from search import find_best_classifier
from sklearn.grid_search import GridSearchCV

options = {
    'C': np.arange(0.2, 5.0, 0.4),
}

search_options = {
    'cv': 3,
    'scoring': 'roc_auc',
    'n_jobs': -1,
    'n_iter': 5,
}

search = RandomizedSearchCV(LinearSVC(), options, verbose=3, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer)  


Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] C=2.2 ...........................................................
[CV] C=2.2 ...........................................................
[CV] C=2.2 ...........................................................
[CV] C=4.2 ...........................................................
[CV] .................................. C=2.2, score=0.953482 -  27.3s
[CV] C=4.2 ...........................................................
[CV] .................................. C=2.2, score=0.897057 -  35.0s
[CV] C=4.2 ...........................................................
[CV] .................................. C=4.2, score=0.953482 -  47.0s
[CV] C=4.6 ...........................................................
[CV] .................................. C=2.2, score=0.809139 -  50.1s
[CV] C=4.6 ...........................................................
[CV] .................................. C=4.2, score=0.900411 -  36.3s
[CV] C=4.6 ......

[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.2min finished


Mejores parámetros: {'C': 4.6000000000000014}


,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
LinearSVC,0.900729,0.878556,0.8751,0.850889,0.878556


# Con Kernel RBF



In [6]:
from search import find_best_classifier
from sklearn.grid_search import GridSearchCV

options = {
    'kernel': ['rbf'],
    'C': np.arange(0.2, 5.0, 0.4),
    'gamma': np.arange(0.01, 1.00, 0.01),
}

search_options = {
    'cv': 3,
    'scoring': 'roc_auc',
    'n_jobs': -1,
    'n_iter': 10,
}

search = RandomizedSearchCV(SVC(), options, verbose=3, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer)  


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] kernel=rbf, C=1.0, gamma=0.75 ...................................
[CV] kernel=rbf, C=1.0, gamma=0.75 ...................................
[CV] kernel=rbf, C=1.0, gamma=0.75 ...................................
[CV] kernel=rbf, C=4.2, gamma=0.15 ...................................
[CV] .......... kernel=rbf, C=1.0, gamma=0.75, score=0.709944 -72.2min
[CV] kernel=rbf, C=4.2, gamma=0.15 ...................................
[CV] .......... kernel=rbf, C=1.0, gamma=0.75, score=0.723079 -77.9min
[CV] kernel=rbf, C=4.2, gamma=0.15 ...................................
[CV] .......... kernel=rbf, C=1.0, gamma=0.75, score=0.650206 -78.2min
[CV] kernel=rbf, C=1.0, gamma=0.82 ...................................
[CV] ......... kernel=rbf, C=4.2, gamma=0.15, score=0.823717 -129.2min
[CV] kernel=rbf, C=1.0, gamma=0.82 ...................................
[CV] .......... kernel=rbf, C=1.0, gamma=0.82, score=0.643071 -80.0min
[CV] kernel=rbf,

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 717.4min finished


Mejores parámetros: {'kernel': 'rbf', 'C': 1.0000000000000002, 'gamma': 0.02}


,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
SVC,0.764366,0.825111,0.843133,0.94,0.825111


# PCA (sólo con Linear SVC)

Hago PCA sobre Linear SVC ya que dio los mejores resultados

In [13]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('pca', PCA()),
    ('clf', LinearSVC()),
])
clf

Pipeline(steps=[('pca', PCA(copy=True, n_components=None, whiten=False)), ('clf', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [ ]:
from helpers import add_prefix

options = {
    'C': np.arange(0.2, 5.0, 0.4),
}

search_options = {
    'cv': 3,
    'scoring': 'roc_auc',
    'n_jobs': -1,
    'n_iter': 30,
}

new_options = add_prefix(options, 'clf__')

no_features = X.shape[1]

new_options.update({
    'pca__n_components' : range(20, no_features, 10)
})

print new_options
print search_options


search = RandomizedSearchCV(clf, new_options, verbose=2, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer)

{'pca__n_components': [20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270], 'clf__C': array([ 0.2,  0.6,  1. ,  1.4,  1.8,  2.2,  2.6,  3. ,  3.4,  3.8,  4.2,
        4.6])}
{'scoring': 'roc_auc', 'n_iter': 30, 'n_jobs': -1, 'cv': 3}
Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] pca__n_components=230, clf__C=0.6 ...............................
[CV] pca__n_components=230, clf__C=0.6 ...............................
[CV] pca__n_components=230, clf__C=0.6 ...............................
[CV] pca__n_components=180, clf__C=0.2 ...............................
[CV] ...................... pca__n_components=180, clf__C=0.2 - 1.4min
[CV] pca__n_components=180, clf__C=0.2 ...............................
[CV] ...................... pca__n_components=230, clf__C=0.6 - 1.6min
[CV] pca__n_components=180, clf__C=0.2 ...............................
[CV] ...................... pca__n_components=230, clf__C=0.6 - 1.6

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 11.2min


[CV] ...................... pca__n_components=180, clf__C=2.6 - 1.5min
[CV] pca__n_components=150, clf__C=1.4 ...............................
[CV] ...................... pca__n_components=180, clf__C=2.6 - 1.5min
[CV] pca__n_components=150, clf__C=1.4 ...............................
[CV] ...................... pca__n_components=180, clf__C=2.6 - 1.5min
[CV] pca__n_components=170, clf__C=3.0 ...............................
[CV] ...................... pca__n_components=150, clf__C=1.4 - 1.3min
[CV] pca__n_components=170, clf__C=3.0 ...............................
[CV] ...................... pca__n_components=150, clf__C=1.4 - 1.2min
[CV] pca__n_components=170, clf__C=3.0 ...............................
[CV] ...................... pca__n_components=150, clf__C=1.4 - 1.2min
[CV] pca__n_components=90, clf__C=3.4 ................................
[CV] ...................... pca__n_components=170, clf__C=3.0 - 1.3min
[CV] pca__n_components=90, clf__C=3.4 ................................
[CV] .

# Hago KNN + PCA acá también

In [3]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from helpers import add_prefix
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('pca', PCA()),
    ('clf', KNeighborsClassifier()),
])
clf

options = {
    'n_neighbors': range(100, 1000),
    'weights': ['distance'],
    'metric' : ['canberra']
    
}


search_options = {
    'cv': 10,
    'scoring': 'roc_auc',
    'n_jobs': -1,
    'n_iter': 40,
}

new_options = add_prefix(options, 'clf__')

no_features = X.shape[1]

new_options.update({
    'pca__n_components' : range(20, no_features, 10)
})

print new_options
print search_options


search = RandomizedSearchCV(clf, new_options, verbose=2, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer) 

{'clf__weights': ['distance'], 'clf__metric': ['canberra'], 'pca__n_components': [20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270], 'clf__n_neighbors': [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254,

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.5min


[CV]  pca__n_components=130, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=291 -  16.5s
[CV] pca__n_components=130, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=291 
[CV]  pca__n_components=130, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=291 -  15.7s
[CV] pca__n_components=130, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=291 
[CV]  pca__n_components=130, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=291 -  16.5s
[CV] pca__n_components=130, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=291 
[CV]  pca__n_components=130, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=291 -  14.9s
[CV] pca__n_components=120, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=152 
[CV]  pca__n_components=130, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=291 -  15.5s
[CV] pca__n_components=120, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=152 
[CV] 

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 13.1min


[CV]  pca__n_components=150, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=361 -  16.1s
[CV] pca__n_components=150, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=361 
[CV]  pca__n_components=150, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=361 -  16.3s
[CV] pca__n_components=150, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=361 
[CV]  pca__n_components=150, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=361 -  15.7s
[CV] pca__n_components=180, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=491 
[CV]  pca__n_components=150, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=361 -  16.3s
[CV] pca__n_components=180, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=491 
[CV]  pca__n_components=150, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=361 -  16.1s
[CV] pca__n_components=180, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=491 
[CV] 

[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 28.2min


[CV]  pca__n_components=260, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=499 -  29.2s
[CV] pca__n_components=80, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=826 
[CV]  pca__n_components=80, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=826 -  10.1s
[CV] pca__n_components=80, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=826 
[CV]  pca__n_components=80, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=826 -  10.6s
[CV] pca__n_components=80, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=826 
[CV]  pca__n_components=80, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=826 -   9.7s
[CV] pca__n_components=80, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=826 
[CV]  pca__n_components=260, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=499 -  27.7s
[CV] pca__n_components=80, clf__metric=canberra, clf__weights=distance, clf__n_neighbors=826 
[CV]  pca__n_

[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 30.1min finished


Mejores parámetros: {'pca__n_components': 200, 'clf__metric': 'canberra', 'clf__weights': 'distance', 'clf__n_neighbors': 152}


,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
Pipeline,0.960858,0.910778,0.905652,0.856444,0.910778
